<div style="text-align: right">CSCI E-7 Introduction to Python Programming for Life Sciences</div>
<div style="text-align: right">Dino Konstantopoulos, 3 February 2019, with some material from Cam Davidson-Pilon</div>

At the end of this notebook, you should have a good understanding of **python list comprehensions**, **python dynamic types**, and be able to use them to improve the look and feel of programs in order to reduce verbosity and minimze the use of looping structures. You should also be comfortable capturing logical arguments into **python predicate functions**.


Please unzip all images from each week's `images.zip` on canvas onto your `C:/Users/<username>/ipynb.images` folder (create it if it doesn't exist). If there's a `data.zip` file on blackboard, unzip its contents onto your `C:/Users/<username>/data` folder (create it if it doesn't exist).

# 1. Probabilities and birthdays

In 1814, Pierre-Simon, marquis de Laplace (23 March 1749 – 5 March 1827), a French scholar whose work was important to the development of mathematics, statistics, physics and astronomy, [wrote](https://en.wikipedia.org/wiki/Classical_definition_of_probability):

>*Probability ... is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible ... when nothing leads us to expect that any one of these cases should occur more than any other.*


Here's some vocabulary:

- **[Experiment](https://en.wikipedia.org/wiki/Experiment_(probability_theory%29):**
  An occurrence with an uncertain outcome that we can observe.
  <br>*For example, rolling a die.*
- **[Outcome](https://en.wikipedia.org/wiki/Outcome_(probability%29):**
  The result of an experiment; one particular state of the world. What Laplace calls a "case."
  <br>*For example:* `4`.
- **[Sample Space](https://en.wikipedia.org/wiki/Sample_space):**
  The set of all possible outcomes for the experiment. 
  <br>*For example,* `{1, 2, 3, 4, 5, 6}`.
- **[Event](https://en.wikipedia.org/wiki/Event_(probability_theory%29):**
  A subset of possible outcomes that together have some property we are interested in.
  <br>*For example, the event "even die roll" is the set of outcomes* `{2, 4, 6}`. 
  
  <br />
<center>
<img src="https://i.warosu.org/data/sci/img/0067/71/1411506402282.png" width="100" height="100" />
</center>

### Coins and Dice

What's the probability of getting heads? How about two heads in a row? How about three heads in a row?

What's the probability of getting a snake eye when rolling one die? How about when rolling two dice? What's the probability of rolling snake eye**s** with two dice? How about a total count of 4?
  
### Birthday Game
What's the probability that someone in this classroom shares your birthday? 
Each person can have your birthday with probability 1/365. There are n−1 people other than yourself, so the probability that someone shares your birthday is ...

Now, what is the probability that any *two* students in this classroom have the *same* birthday? Which one of the two you think is higher?

### Probability that someone in this classroom shares your birthday? 

Each person can have your birthday with probability 1/365. There are n-1 people other than yourself, so the probability that someone shares your birthday is (n-1)/ 365. 

Why do we add probabilities? Because it's the **union** of two possibilities, as long as they have no intersection.

Actually, if they have an intersection:
$$p(A + B) = p(A) + p(B) - p(A \cap B)$$

In [23]:
# assume 23 people in class
def probSomeoneShares(people):
    return (people - 1)/365

probSomeoneShares(23)

0.06027397260273973

Hmm.. What happens if there are 366 people in class?

In [24]:
probSomeoneShares(366)

1.0

Really?

Remember how we assumed birthdays are independent? Well, they aren’t!

If Person A and Person B have the same birthday, that probability is indeed $\frac{1}{365}$. But if we bring in a Person C to increase the probability of matching Person A, that person's birthday may also match person B's birthday! So according to the formula above, we need to remove the intersection. As we bring in more and more people, we need to remove all possible intersections. So as we bring in two people to match Person A, we need to remove the probability of these two people simultaneously matching A, so we need to remove $(\frac{1}{365})^2$. As we bring in a third person to match person A, we will need to remove the probability of the third matching the first and the third matching the second, and the third matching both! What a mess!

There is a better formula to take advantage of:
$$p(A \cup B) = p(A) * p(B)$$

So we should compare everyone to everyone else by evaluating the chance of 2 people having *different* birthdays! That is

$$ 1 - \frac{1}{365} = \frac{364}{365} = .997$$

The chance of your friend having a different birthday than *you* is:

$$ 1 - \frac{1}{365} = \frac{364}{365} = .997$$

What's the probability of getting heads when flipping a coin? What's the probability of getting heads twice in a row?

Now, what is the chance of 2 people having different birthdays than you? The product of each, since it's an intersection, exactly like a coin flip!

$$ (1 - \frac{1}{365})^2$$

And so the chance of $n$ people having different birthdays than you must be:

$$ (1 - \frac{1}{365})^n$$

And so the chance of people having different birthdays than you in a classroom of $n$ people must be:

$$ (1 - \frac{1}{365})^{n-1}$$

And so the chance of people in a classroom of $n$ people having the the *same* birthday than you must be 1 minus the previous:

$$ 1 - (1 - \frac{1}{365})^{n-1}$$

Write this down as a python expression.

In [21]:
# assume 23 people in class
1 - (1 - 1./365.)**22

0.058571325264343166

and now..

In [25]:
# assume 366 people in class
1 - (1 - 1./365.)**366

0.6336315859687802

### Probability that two students in this classroom have the same birthday? 

For 2 students to share a birthday, we will approach the problem the other way around too, because probability that no two people have the same birthday is easier to ﬁnd.

Let A be the event that two people have the same birthday. Then Ac is the event that no two people have the same birthday. Note that P(A) = 1−P(Ac).

We start with person 1; this person can have any 1 of 365 days out of the year.

A second person can only have a birthday on the 364 days out of the year that hasn’t been ‘taken
By assumption of random birthdays, and of uniform probability, the chance that this person has any of the 364 birthdays is 364/365.

A third person can only have a birthday out of the 353 days not ‘taken’ and the corresponding probability of such an event is 363/365.

This continues until we’ve covered all n people.

Since this is an **intersection** of events, probabilities multiply.

So write down the probability of any two people sharing a birthday in a classroom of 223 people..

In [1]:
from operator import mul
from functools import reduce

# assume 23 people in class
def prob2StudentsShare(people):
    """return 1 minus the negation (students NOT sharing a common birthday)"""
    """return 1 - (365 * 364 * 363 * 362 * 361 * 360 * 359 * 358 * 357 * 356 * ...)/ (365 ** 23)"""
    l = [n for n in range(365, 365 - people, -1)]
    return 1 - (reduce(mul, l, 1) / (365 ** people))

print("2 share: " + str(prob2StudentsShare(23)))

2 share: 0.5072972343239854


What? With 23 people in a classroom (you are more than 23 right now!), there are higher than 50/50 odds that two of you share the same birthday! Try it out during class break!

Why is that?

Because with 23 people we can form 253 pairs: 

$$\frac{23 * 22}{2} = 253$$ 

Once again, the chance of 2 people having *different* birthdays is:

$$ 1 - \frac{1}{365} = \frac{364}{365} = .997$$

Making 253 comparisons and having them all be different is like getting one of heads or tails 253 times in a row:

$$(\frac{364}{365})^{253} = .499$$

So the chance we find a match is: 1 – 49.95% = 50.05%, or just over half.

The probability of a match for any number of people $n$ the formula is:

$$1 - (\frac{364}{365})^{n(n-1)/2}$$

In fact, $\sqrt{n}$ is roughly the number you need to have a 50% chance of a match with $n$ items.

### Note 

For n = 30, the odds of a common birthday increase to 70.6%, and most people still find it hard to believe that among 30 people there are probably two who have the same birthday! The table below lists various values of n and the probabilities, 1 − Pn, that at least two people have a common birthday.

|n |10| 20| 23| 30| 50| 60| 70| 100|
|--|--|--|--|--|--|--|--|--|--|--|
|1 − Pn| 11.7%| 41.1%| 50.7%| 70.6%| 97.0%| 99.4%| 99.92%| 99.9994%|

# 2. Writing the Probability function in Python

We need to introduce three definitions which are very important to life sciences:

* [Frequency](https://en.wikipedia.org/wiki/Frequency_%28statistics%29): a number describing how often an outcome occurs. Can be a count like 121801, or a ratio like 0.515.

* [Distribution](http://mathworld.wolfram.com/StatisticalDistribution.html): A mapping from outcome to frequency for each possible outcome in a sample space. 

* [Probability Distribution](https://en.wikipedia.org/wiki/Probability_distribution): The distribution above, which has been *normalized* so that the sum of the frequencies is 1.



Now that we've warmed up with birthdays, let's see if we can define a probability function in Python

<br />
<center>
<img src="https://i.warosu.org/data/sci/img/0067/71/1411506402282.png" width="100" height="100" />
</center>

`p` is the traditional name give to the Probability function. Use Laplace's definition above to write the probability function `p`.

```python
from fractions import Fraction
def p(event, space): 
    "The probability of an event, given a sample space of equiprobable outcomes."
    return Fraction(len(event & space), 
                    len(space))
```
Note:
* We use ```Fraction``` rather than regular division because I want exact answers like 1/3, not 0.3333333333333333.
* `&` is the python set *intersection* operation, while `|` is the python *union* operation.

In [1]:
from fractions import Fraction
def p(event, space): 
    "The probability of an event, given a sample space of equiprobable outcomes."
    return Fraction(len(event & space), #& is an intersection 
                    len(space))

**Exercise**: What's the probability of rolling an even number with a single six-sided fair die? Use python tuples (unordered collection with no duplicate elements), since we don't expect them to change.

Define the sample space D:
```D    = {...}```

and the event even:
```even = {...}```

and compute the probability:
```p(even, D)```

Copy and paste the code above in the cell below, and replace ```...``` with the right values!

In [21]:
from fractions import Fraction
def p(event, space): 
    "The probability of an event, given a sample space of equiprobable outcomes."
    return Fraction(len(event & space), 
                    len(space))

D    = {1,2,3,4,5,6}
even = {2,4,6}
p(even, D)

Fraction(1, 2)

In [ ]:
even = {2, 4, 6, 8, 10, 12, 14, 16, 18, 20}
p(even, D)

What happens if you specify ```even = {2, 4, 6, 8, 10, 12, 14, 16, 18, 20}```?

To note:
* The definition of ```p``` uses ```len(event & space)``` rather than ```len(event)``` because I don't want to count outcomes that were specified in event but aren't actually in the sample space.

# 3. Working with transformations instead of samples

<br />
<center>
<img src="http://agilitrix.com/wp-content/uploads/2013/02/1-Transformation.jpg"  width="500" />
</center>

Sometimes we don't have a straightforward way to easily enumerate all possible samples in a sample space, but we have an easy way of defining a transformation that will yield a desired sample. In other words, we want to work with **lambdas** instead of **objects**. 

Here's a generator for natural numbers, a compact, transformation-based way of defining natural numbers so that we don't have to write a lot of data:

In [ ]:
def numbers():
    i = 0
    while True:
        yield i
        i += 1

And here's how to consume the generator (a bit ugly, granted, can you improve on this, i.e. no `break` statement?):

In [ ]:
import queue as Q
q = Q.Queue()
for index, item in enumerate(numbers()):
    q.put(item)
    if index == 100:
        break
print(list(q.queue))

In the case of the die roll, I can define an `even` lambda: 

In [ ]:
def even(n): return n % 2 == 0

Now in order to make `p(even, D)` work, I'll have to modify `p` to accept an event as either
a *set* of outcomes (as before), or a *predicate* over outcomes&mdash;a function that returns true for an outcome that is in the event:

# 4. Extending the probability function with polymorphic types

Let's define `ProbDist`, a probability distribution, to take the same kinds of arguments that dict does: a set of (key, val) pairs. This is the first time (in class), that we define a Python `class`, instead of a Python function/lambda. That is why we define its constructor `__init__()`. We assume `self` (`this` in Python) is composed of a set. All `ProbDist` does is iterate over a dicitonary and convert count values into fractions that add up to 1 so that they can be probabilities of a space of possible events (where the probability of each event adds up to 1).

In [4]:
class ProbDist(dict):
    """A Probability Distribution; an {outcome: probability} mapping."""
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        # Make probabilities sum to 1.0; assert no negative probabilities
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0

Crap! What does `**kwargs` mean? Look at this function below. It's a magic function that returns all positional arguments and all names arguments.

In [33]:
def magic(*args, **kwargs):
    print ("unnamed args: ", args)
    print ("keyword args: ", kwargs)

So we can do:

In [32]:
magic(1, 2, 3, key1 = 'Harvard', key2 = 'Hockey', key3 = "rocks!")

unnamed args:  (1, 2, 3)
keyword args:  {'key1': 'Harvard', 'key2': 'Hockey', 'key3': 'rocks!'}


See how it gives us all positional and all named arguments?

Now we need to modify the function `p` to accept either a collection as the first argument, as we had previously, **or a predicate function instead**. We say that we make the function **polymorphic** because it can now accept different types of arguments.

But we need a `such_that()` function to do this, which converts predicate functions to real collections of data.

In [7]:
def p(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes.
    event can be either a set of outcomes, or a predicate (true for outcomes in the event)."""
    if is_predicate(event):
        event = such_that(event, space)
    return Fraction(len(event & space), len(space))

is_predicate = callable

def such_that(predicate, collection): 
    """The subset of elements in the collection for which the predicate is true."""
    return {e for e in collection if predicate(e)}

Now, let's work with M&M bags.

# 5. M&Ms
<br />
<center>
<img src="https://upload.wikimedia.org/wikipedia/en/9/97/M%26M_spokescandies.jpeg" />
</center>

Here's a classic urn problem (or "bag" problem) from prolific Python/Probability author [Allen Downey](http://allendowney.blogspot.com/), which also happens to be a classic interview question:

> The blue M&M was introduced in 1995.  Before then, the color mix in a bag of plain M&Ms was (30% Brown, 20% Yellow, 20% Red, 10% Green, 10% Orange, 10% Tan).  Afterward it was (24% Blue , 20% Green, 16% Orange, 14% Yellow, 13% Red, 13% Brown). 
A friend of mine has two bags of M&Ms, and he tells me that one is from 1994 and one from 1996.  He won't tell me which is which, but he gives me one M&M from each bag.  One is yellow and one is green.  What is the probability that the yellow M&M came from the 1994 bag? Well, the old M&M bags' yellow count was higher, so it must be higher, right? But how to count?

To solve this problem, we'll first represent probability distributions for each bag: `bag94` and `bag96`, by using `ProbDist` and passing in dictionaries for each year:
```python
bag94 = ProbDist(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
bag96 = ProbDist(...)  #fill this in, please
```

We want to convert the dictionary into a dictionary where the values are probabilities, so that means all the values need to sum to 1. So we apply the `ProbDist` function to the input dictionary.

In [29]:
bag94 = ProbDist(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
bag96 = ProbDist(brown=13, yellow=14, red=13, green=20, orange=14, blue=24)

In [31]:
bag96

{'brown': 0.1326530612244898,
 'yellow': 0.14285714285714285,
 'red': 0.1326530612244898,
 'green': 0.20408163265306123,
 'orange': 0.14285714285714285,
 'blue': 0.24489795918367346}

Sample the collections:

In [12]:
import random
random.sample(list(bag96), 3)

['green', 'red', 'orange']

Now define the following predicate function (cut and paste into cell below):

```python
def blue(outcome): return 'blue' in outcome
```

In [16]:
def blue(outcome): return 'blue' in outcome

Now use the `blue` predicate function to filter only the blue M&Ms from each bag of M&Ms, pre-95 and post-95. 
```python
such_that(...) # fill this in
```

In [17]:
bag_94_blue = such_that(blue, {'brown', 'yellow', 'red', 'green', 'orange', 'tan'})
bag_94_blue

set()

It's the empty set!

In [15]:
bag_96_blue = such_that(blue, {'blue', 'green', 'orange', 'yellow', 'red', 'brown'})
bag_96_blue

{'blue'}

That's the only M&M which passes the predicate function `blue`!

Now I can write:

In [24]:
p({'blue'}, {'blue', 'green', 'orange', 'yellow', 'red', 'brown'})

Fraction(1, 6)

But also:

In [25]:
p(blue, {'blue', 'green', 'orange', 'yellow', 'red', 'brown'})

Fraction(1, 6)

In the first case, I get the probability of getting a blue M&M by specifying the collection of interest, and in the second case exactly the same thing by specifying a filtering predicate.

## Homework

Now use what you learned from strings and predicate functions to filter each M&M bag collection so that you only return the M&Ms **whose color starts with the letter 'b'**. Fill in the ellipses below.

In [41]:
def starts_with_letter_b(outcome): return outcome.startswith('b')

In [46]:
bag_94_starts_with_letter_b = such_that(starts_with_letter_b, {'brown', 'yellow', 'red', 'green', 'orange', 'tan'})
bag_94_starts_with_letter_b

{'brown'}

In [47]:
bag_96_starts_with_letter_b = such_that(starts_with_letter_b, {'blue', 'green', 'orange', 'yellow', 'red', 'brown'})
bag_96_starts_with_letter_b

{'blue', 'brown'}

Hint: a string that starts with the letter 's' can be be predicated this way:
```python
'supercalifragilisticespielidocious'.startswith('s')
```

In [36]:
'supercalifragilisticespielidocious'.startswith('s')

True